In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/movielense20m/rating.csv
/kaggle/input/movielense20m/movie.csv


# USER BASED

### 1. PREPARING DATASET - CREATING USER MOVIE DF

In [3]:
def create_user_movie_df():
    import pandas as pd
    movies = pd.read_csv('../input/movielense20m/movie.csv', low_memory=False)
    ratings = pd.read_csv('../input/movielense20m/rating.csv')
    df = movies.merge(ratings, how="left", on="movieId")
    df['title'] = df.title.str.replace('(\(\d\d\d\d\))', '')
    df['title'] = df['title'].apply(lambda x: x.strip())
    a = pd.DataFrame(df["title"].value_counts())
    rare_movies = a[a["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()
user_movie_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


title,"'burbs, The",(500) Days of Summer,*batteries not included,...And Justice for All,10 Things I Hate About You,"10,000 BC",101 Dalmatians,101 Dalmatians (One Hundred and One Dalmatians),102 Dalmatians,12 Angry Men,...,Zero Dark Thirty,Zero Effect,Zodiac,Zombieland,Zoolander,Zulu,[REC],eXistenZ,xXx,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
user_movie_df.shape

(138493, 3134)

In [5]:
user_movie_df.columns

Index([''burbs, The', '(500) Days of Summer', '*batteries not included',
       '...And Justice for All', '10 Things I Hate About You', '10,000 BC',
       '101 Dalmatians', '101 Dalmatians (One Hundred and One Dalmatians)',
       '102 Dalmatians', '12 Angry Men',
       ...
       'Zero Dark Thirty', 'Zero Effect', 'Zodiac', 'Zombieland', 'Zoolander',
       'Zulu', '[REC]', 'eXistenZ', 'xXx', '¡Three Amigos!'],
      dtype='object', name='title', length=3134)

In [6]:
len(user_movie_df.columns)

3134

In [9]:
movie = user_movie_df["Matrix, The"]
movie

userId
1.0         NaN
2.0         NaN
3.0         5.0
4.0         NaN
5.0         NaN
           ... 
138489.0    4.0
138490.0    NaN
138491.0    NaN
138492.0    5.0
138493.0    4.5
Name: Matrix, The, Length: 138493, dtype: float64


### 2. DETERMINING THE WATCHED MOVIES BY THE 108170 USER_ID

In [13]:
user_id = 108170
user_id_df = user_movie_df[user_movie_df.index == user_id]
user_id_df.head()

title,"'burbs, The",(500) Days of Summer,*batteries not included,...And Justice for All,10 Things I Hate About You,"10,000 BC",101 Dalmatians,101 Dalmatians (One Hundred and One Dalmatians),102 Dalmatians,12 Angry Men,...,Zero Dark Thirty,Zero Effect,Zodiac,Zombieland,Zoolander,Zulu,[REC],eXistenZ,xXx,¡Three Amigos!
userId,,,,,,,,,,,,,,,,,,,,,
108170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
movies_watched = user_id_df.columns[user_id_df.notna().any()].tolist()
len(movies_watched)

186

### 3. GETTING OTHER USERS WATCHING THE SAME MOVIES

In [15]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()

title,2001: A Space Odyssey,"Adventures of Priscilla, Queen of the Desert, The",Akira,Aladdin,Aliens,Almost Famous,Along Came Polly,Amadeus,American History X,Any Given Sunday,...,"Untouchables, The","Usual Suspects, The",Wallace & Gromit: A Close Shave,Wallace & Gromit: The Best of Aardman Animation,Wallace & Gromit: The Wrong Trousers,"Wedding Singer, The",Who Framed Roger Rabbit?,Wild at Heart,Willow,X2: X-Men United
userId,,,,,,,,,,,,,,,,,,,,,
1.0,3.5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,3.5,3.5,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0
2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,3.0,4.0,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
movies_watched_df.shape

(138493, 186)

In [17]:
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count.head()

,userId,movie_count
0,1.0,54
1,2.0,11
2,3.0,47
3,4.0,5
4,5.0,16


In [19]:
# those who watched at least 60% of the movies that this 108170 user id watched
perc = len(movies_watched) * 60 / 100
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
users_same_movies.head()

90      91.0
115    116.0
155    156.0
293    294.0
297    298.0
Name: userId, dtype: float64

### 4. DETERMINING THE MOST SIMILAR USERS TO 108170 USER ID

In [24]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies.index)], user_id_df[movies_watched]])
final_df.head()

title,2001: A Space Odyssey,"Adventures of Priscilla, Queen of the Desert, The",Akira,Aladdin,Aliens,Almost Famous,Along Came Polly,Amadeus,American History X,Any Given Sunday,...,"Untouchables, The","Usual Suspects, The",Wallace & Gromit: A Close Shave,Wallace & Gromit: The Best of Aardman Animation,Wallace & Gromit: The Wrong Trousers,"Wedding Singer, The",Who Framed Roger Rabbit?,Wild at Heart,Willow,X2: X-Men United
userId,,,,,,,,,,,,,,,,,,,,,
90.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.5,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,4.5
115.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
293.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
final_df.T.corr()

userId,90.0,115.0,155.0,293.0,297.0,358.0,366.0,387.0,585.0,613.0,...,137838.0,137853.0,137884.0,137992.0,138133.0,138207.0,138253.0,138324.0,138410.0,108170.0
userId,,,,,,,,,,,,,,,,,,,,,
90.0,1.000000,-0.072169,-0.466387,NaN,-0.570352,-0.594089,-0.641689,2.740985e-01,0.370479,NaN,...,-0.047796,-0.150516,0.738549,-0.206349,NaN,0.351330,-0.707107,NaN,-0.301511,-0.049790
115.0,-0.072169,1.000000,0.821584,NaN,1.000000,NaN,NaN,-9.654792e-17,-1.000000,NaN,...,0.097839,0.950586,NaN,0.548821,NaN,0.263523,NaN,NaN,NaN,0.744901
155.0,-0.466387,0.821584,1.000000,NaN,NaN,0.647059,NaN,-3.830654e-01,-0.654654,NaN,...,-0.079736,0.716419,NaN,0.116335,NaN,0.577350,1.000000,NaN,NaN,0.616206
293.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297.0,-0.570352,1.000000,NaN,NaN,1.000000,NaN,NaN,9.432422e-01,NaN,NaN,...,0.402524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.524500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138207.0,0.351330,0.263523,0.577350,NaN,NaN,NaN,NaN,-3.699700e-01,NaN,NaN,...,0.126655,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,0.671574
138253.0,-0.707107,NaN,1.000000,NaN,NaN,-0.500000,0.484881,-8.528029e-01,NaN,NaN,...,-0.294166,0.636364,NaN,0.371154,NaN,NaN,1.000000,NaN,0.133631,0.282451
138324.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.188982,NaN,NaN,1.000000,NaN,NaN,NaN,1.0,NaN,0.500000


In [26]:
# Correlation Df
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df.head()

,user_id_1,user_id_2,corr
0,78461.0,129513.0,-1.0
1,69792.0,125024.0,-1.0
2,71790.0,70815.0,-1.0
3,54258.0,101419.0,-1.0
4,78086.0,64532.0,-1.0


In [28]:
# TOP SIMILAR USERS
top_users = corr_df[(corr_df["user_id_1"] == user_id) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)
top_users.head()

,userId,corr
57,25613.0,0.980953
56,135521.0,0.970148
55,80770.0,0.968004
54,92635.0,0.933859
53,23164.0,0.933848


### 5. CALCULATING WEIGHTED RATING

In [30]:
ratings = pd.read_csv('../input/movielense20m/rating.csv')
top_users_ratings = top_users.merge(ratings[["userId", "movieId", "rating"]], how='inner')
top_users_ratings.head()

,userId,corr,movieId,rating
0,25613.0,0.980953,32,5.0
1,25613.0,0.980953,145,3.0
2,25613.0,0.980953,196,1.0
3,25613.0,0.980953,466,3.0
4,25613.0,0.980953,552,0.5


In [32]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.head()

,userId,corr,movieId,rating,weighted_rating
0,25613.0,0.980953,32,5.0,4.904766
1,25613.0,0.980953,145,3.0,2.942859
2,25613.0,0.980953,196,1.0,0.980953
3,25613.0,0.980953,466,3.0,2.942859
4,25613.0,0.980953,552,0.5,0.490477


In [33]:
temp = top_users_ratings.groupby('movieId').sum()[['corr', 'weighted_rating']]
temp.columns = ['sum_corr', 'sum_weighted_rating']
temp.head()

,sum_corr,sum_weighted_rating
movieId,,
1,9.359345,27.207460
2,5.505255,17.143026
3,2.474102,8.890035
5,0.730575,2.191725
6,5.989265,24.185020


In [34]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted_average_recommendation_score'] = temp['sum_weighted_rating'] / temp['sum_corr']
recommendation_df['movieId'] = temp.index
recommendation_df = recommendation_df.sort_values(by='weighted_average_recommendation_score', ascending=False)
recommendation_df.head(30)

,weighted_average_recommendation_score,movieId
movieId,,
1747,5.0,1747
4167,5.0,4167
994,5.0,994
2664,5.0,2664
8207,5.0,8207
8228,5.0,8228
2799,5.0,2799
8533,5.0,8533
8656,5.0,8656


In [37]:
movies = pd.read_csv('../input/movielense20m/movie.csv', low_memory=False)
movies_from_user_based = movies.loc[movies['movieId'].isin(recommendation_df['movieId'].head(10))]['title']
movies_from_user_based.head(30)
movies_from_user_based[:5].values

array(['Foxfire (1996)', 'Big Night (1996)', 'Wag the Dog (1997)',
       'Invasion of the Body Snatchers (1956)', 'Problem Child 2 (1991)'],
      dtype=object)

# ITEM BASED

In [39]:
movie_id = ratings[(ratings["userId"] == user_id) & (ratings["rating"] == 5.0)]. \
    sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]
movie_id

7044

In [41]:
movie_title = movies[movies["movieId"] == movie_id]["title"].str.replace('(\(\d\d\d\d\))', '').str.strip().values[0]
movie_title

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


'Wild at Heart'

### 1. ITEM - BASED FILM SUGGESTIONS BASED ON CORRELATION

In [42]:
def item_based_recommender(movie_name, user_movie_df):
    movie = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie).sort_values(ascending=False).head(10)

item_based_recommender(movie_title, user_movie_df)

title
Wild at Heart                     1.000000
My Science Project                0.570187
Mediterraneo                      0.538868
National Lampoon's Senior Trip    0.533029
Old Man and the Sea, The          0.503236
Cashback                          0.497123
Clockwatchers                     0.483337
Repo Man                          0.478674
Lost Highway                      0.476251
Blue Velvet                       0.471225
dtype: float64

# HYBRID RECOMMENDATION

In [44]:
recommendation_10 = pd.DataFrame()
recommendation_10['user_based_recommendations'] = movies_from_user_based[:5].values.tolist()
recommendation_10['item_based_recommendations'] = item_based_recommender(movie_title, user_movie_df)[1:6].index

recommendation_10

,user_based_recommendations,item_based_recommendations
0,Foxfire (1996),My Science Project
1,Big Night (1996),Mediterraneo
2,Wag the Dog (1997),National Lampoon's Senior Trip
3,Invasion of the Body Snatchers (1956),"Old Man and the Sea, The"
4,Problem Child 2 (1991),Cashback
